In [6]:
import pandas as pd
import numpy as np
from pprint import pprint

### Cчитывание баз данных по московским станциям метро и автобусным остановкам

In [7]:
metro_station_base = pd.read_excel('C:/Projects/LP_Lessons/lesson3/repair_of_escalators/repair_of_escalators.xlsx', encoding = 'cp1251')
bus_stop_base = pd.read_excel('C:/Projects/LP_Lessons/lesson3/bus_stop_list/bus_stop_list.xlsx', encoding='cp1251')

### Транспонирование 

In [8]:
msb = metro_station_base.T
bsb = bus_stop_base.T

### Функция расчета расстояния между точками

In [9]:
def distance(coord_1, coord_2):

    r_earth = 6371000

    long_1 = coord_1[0]*np.pi/180
    lati_1 = coord_1[1]*np.pi/180

    long_2 = coord_2[0]*np.pi/180
    lati_2 = coord_2[1]*np.pi/180

    d_x = np.sin(long_1 - long_2)
    d_y = np.sin(lati_1 - lati_2)

    # точная оценка
    d_rad = np.arccos( np.sin(long_1) * np.sin(long_2)
            + np.cos(long_1) * np.cos(long_2) * np.cos(lati_1 - lati_2))
    
    dist = d_rad * r_earth
    
    return dist

### Функция для суммирования количества автобусных остановок.

In [10]:
def close_plus(d, dots, rad):
    '''
    функция возвращает 1 если точка находится ближе чем rad хотя бы к одной из точек в списке точек dots, иначе 0
    '''
    answ = False
    # d - георгафическая координата выхода автобусной остановки
    # dots - несколько географических координат выходов из метро
    for dot in dots: # dot - географические кооридинаты выхода из метро
        
        cond = distance(d, dot) < rad
        answ = int(answ or cond)

    return int(answ)

### Создание словаря со списком названий станций метро и координат выходов с этой станции

In [16]:
def main_calc(ms, bs, rad):
    
    len_bsb = len(bs.T)
    len_msb = len(ms.T)
    
    # создание словаря с ключом из названий станций метро и значениями координат всех выходов в виде списка из пар координта до долготе и широте
    ms_list = {}
    for i in range(len_msb):
        print('#', end = '')
        cond_1 = ms[i]['NameOfStation'] in ms_list
        if not cond_1:
            ms_list[ms[i]['NameOfStation']] = [[ms[i]['Longitude_WGS84'], ms[i]['Latitude_WGS84']], ]
        else: 
            ms_list[ms[i]['NameOfStation']].append([ms[i]['Longitude_WGS84'], ms[i]['Latitude_WGS84']])
    
    print('\n')
    print('Создан словарь станций метро и координат выходов.')
    
    # создание словаря с ключами из названий станций метро и значениями числа автобусных остановок
    ms_list_count = {}
    for ms_name, ms_dots in ms_list.items():
        
        print('#', end = '')
        
        for j in range(len_bsb):
            dot = [bs[j]['Longitude_WGS84'], bs[j]['Latitude_WGS84'] ]
            if ms_name not in ms_list_count:
                ms_list_count[ms_name] = close_plus(dot, ms_dots, rad)
            else:
                ms_list_count[ms_name] += close_plus(dot, ms_dots, rad)
    
    print('\n')
    print('Счет завершен.')
    
    del ms_list
    
    return ms_list_count

In [17]:
radius = 500
mc = main_calc(msb, bsb, radius)

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [18]:
def  main_select(d):
    max = 0
    for key, value in d.items():
        if value > max: 
            max = value
            d = {key: max}
    pprint(d)
    return d

In [19]:
main_select(mc)

{'Новогиреево': 33}


{'Новогиреево': 33}